In [1]:
import numpy as np

In [2]:
matches_query = np.load('data/matches_query.npy', allow_pickle=True).item()
len(matches_query)

5

In [3]:
scene_fingerprints = np.load('data/scene_fingerprints.npy')

In [29]:
scene_fingerprints[scene_fingerprints[:, 1] == 14154]
#492 - 486
#len(matches_query['aggregations']['scenes']['buckets'])

array([[  158, 14154]])

In [65]:
matches_query['aggregations']['scenes']['buckets'][1] #['top_hits']['hits']['hits']

{'key': 'QpwOvHQBdT-cPgIEckxS',
 'doc_count': 278,
 'score': 0.24601769911504426,
 'bg_count': 1130,
 'top_hits': {'hits': {'total': {'value': 278, 'relation': 'eq'},
   'max_score': 0.0,
   'hits': [{'_index': 'fingerprints_patterns',
     '_type': '_doc',
     '_id': 'TgZxwHQBWmCctFZv2NOd',
     '_score': 0.0,
     '_source': {'offset': 875, 'hash': 8235}},
    {'_index': 'fingerprints_patterns',
     '_type': '_doc',
     '_id': 'WgZxwHQBWmCctFZv2NOd',
     '_score': 0.0,
     '_source': {'offset': 891, 'hash': 658665}},
    {'_index': 'fingerprints_patterns',
     '_type': '_doc',
     '_id': 'oAZxwHQBWmCctFZv2NOd',
     '_score': 0.0,
     '_source': {'offset': 948, 'hash': 692456}},
    {'_index': 'fingerprints_patterns',
     '_type': '_doc',
     '_id': 'rgZxwHQBWmCctFZv2NOd',
     '_score': 0.0,
     '_source': {'offset': 960, 'hash': 13331}},
    {'_index': 'fingerprints_patterns',
     '_type': '_doc',
     '_id': 'vQZxwHQBWmCctFZv2NOd',
     '_score': 0.0,
     '_source': {

In [83]:
def score_with_offset(matches_query, fingerprints):
    """
    First version of the exercise. Searches the most frequent offset difference
    for each element in matches_query
    """
    result = []
    for candidates in matches_query['aggregations']['scenes']['buckets']:
        key = candidates['key']
        freq = {} #in this dictionary we'll have the frequency of the difference between offsets
        for hit in candidates['top_hits']['hits']['hits']:
            match_hash = hit['_source']['hash']
            match_offset = hit['_source']['offset']
            possible_offsets = scene_fingerprints[scene_fingerprints[:, 1] == match_hash][:,0] - match_offset
            intersec_offset = set(freq.keys()).intersection(possible_offsets)
            if len(intersec_offset) > 1:
                subset_freq = dict((k, freq[k]) for k in intersec_offset if k in freq)
                delta_offset = max(subset_freq, key=subset_freq.get)
            elif len(intersec_offset) == 1:
                delta_offset = list(intersec_offset)[0]
            else:
                delta_offset = min(possible_offsets, key=abs)
            #if key == '5ErZpXQB1fGddnL2pC6x':
            #    print(possible_offsets, delta_offset)
            if delta_offset in freq:
                freq[delta_offset] += 1
            else:
                freq[delta_offset] = 1
        offset = max(freq, key=freq.get)
        offset_count = freq[offset]
        offset_score = offset_count/candidates['bg_count']
        result.append({'key' : key, 'offset' : offset, 'offset_count' : offset_count,
                       'offset_score' : offset_score })
               
    return result
        
score_with_offset(matches_query, scene_fingerprints)
            

[{'key': 'DjvCu3QBoaf2Y_2A3bmT',
  'offset': 6,
  'offset_count': 220,
  'offset_score': 0.18410041841004185},
 {'key': 'QpwOvHQBdT-cPgIEckxS',
  'offset': 15,
  'offset_count': 205,
  'offset_score': 0.18141592920353983},
 {'key': '1MtUpXQBXtu81SlS8nsP',
  'offset': 244,
  'offset_count': 1,
  'offset_score': 0.006329113924050633},
 {'key': 'lYJFpHQBOaObx7xaWbv5',
  'offset': 391,
  'offset_count': 1,
  'offset_score': 0.005681818181818182},
 {'key': 'WkEslXQBkSohu0M7zw3h',
  'offset': -18,
  'offset_count': 2,
  'offset_score': 0.0035026269702276708},
 {'key': 'lWGJqXQBE5vHt1SDbQAU',
  'offset': 386,
  'offset_count': 3,
  'offset_score': 0.0027447392497712718},
 {'key': 'epIdqHQB3dj8gDV29z8G',
  'offset': 287,
  'offset_count': 1,
  'offset_score': 0.005813953488372093},
 {'key': 'oR28onQBMd_8Sl4jtDNj',
  'offset': 87,
  'offset_count': 2,
  'offset_score': 0.003780718336483932},
 {'key': '5ErZpXQB1fGddnL2pC6x',
  'offset': -302,
  'offset_count': 4,
  'offset_score': 0.007722007722

In [75]:

def calc_freq(freq, possible_offsets):
    """
    I use an intersection to determine if delta_offset exists in freq,
    if there's more than one option, I use the one with maximum frequency
    """
    intersec_offset = set(freq.keys()).intersection(possible_offsets)
    if len(intersec_offset) > 1:
        subset_freq = dict((k, freq[k]) for k in intersec_offset if k in freq)
        delta_offset = max(subset_freq, key=subset_freq.get)
    elif len(intersec_offset) == 1:
        delta_offset = list(intersec_offset)[0]
    else:
        delta_offset = min(possible_offsets, key=abs)
    if delta_offset in freq:
        freq[delta_offset] += 1
    else:
        freq[delta_offset] = 1
    return freq



def score_with_offset2(matches_query, fingerprints):
    """
    In this version I use the function calc_freq to update the value 
    of the dictionary saving the frequencies of the difference between offsets
    """
    result = []
    for candidates in matches_query['aggregations']['scenes']['buckets']:
        key = candidates['key']
        freq = {}
        for hit in candidates['top_hits']['hits']['hits']:
            match_hash = hit['_source']['hash']
            match_offset = hit['_source']['offset']
            possible_offsets = scene_fingerprints[scene_fingerprints[:, 1] == match_hash][:,0] - match_offset
            freq = calc_freq(freq, possible_offsets)
        offset = max(freq, key=freq.get)
        offset_count = freq[offset]
        offset_score = offset_count/candidates['bg_count']
        result.append({'key' : key, 'offset' : offset, 'offset_count' : offset_count,
                       'offset_score' : offset_score })
               
    return result
        
score_with_offset2(matches_query, scene_fingerprints)

I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!
I'm here!


[{'key': 'DjvCu3QBoaf2Y_2A3bmT',
  'offset': 6,
  'offset_count': 220,
  'offset_score': 0.18410041841004185},
 {'key': 'QpwOvHQBdT-cPgIEckxS',
  'offset': 15,
  'offset_count': 205,
  'offset_score': 0.18141592920353983},
 {'key': '1MtUpXQBXtu81SlS8nsP',
  'offset': 244,
  'offset_count': 1,
  'offset_score': 0.006329113924050633},
 {'key': 'lYJFpHQBOaObx7xaWbv5',
  'offset': 391,
  'offset_count': 1,
  'offset_score': 0.005681818181818182},
 {'key': 'WkEslXQBkSohu0M7zw3h',
  'offset': -18,
  'offset_count': 2,
  'offset_score': 0.0035026269702276708},
 {'key': 'lWGJqXQBE5vHt1SDbQAU',
  'offset': 386,
  'offset_count': 3,
  'offset_score': 0.0027447392497712718},
 {'key': 'epIdqHQB3dj8gDV29z8G',
  'offset': 287,
  'offset_count': 1,
  'offset_score': 0.005813953488372093},
 {'key': 'oR28onQBMd_8Sl4jtDNj',
  'offset': 87,
  'offset_count': 2,
  'offset_score': 0.003780718336483932},
 {'key': '5ErZpXQB1fGddnL2pC6x',
  'offset': -302,
  'offset_count': 4,
  'offset_score': 0.007722007722

In [79]:
def calc_freq_alt(freq, possible_offsets):
    """
    We calculate the frequency of the difference between all the offsets for a given hash
    the max value shouldn't change in the final result.
    """
    for off in possible_offsets:
        if off in freq:
            freq[off] += 1
        else:
            freq[off] = 1
    return freq


def score_with_offset_alt(matches_query, fingerprints):
    """
    This is the same as the prior version of the exercise, the only
    difference is the calc_freq function
    """
    result = []
    for candidates in matches_query['aggregations']['scenes']['buckets']:
        key = candidates['key']
        freq = {}
        for hit in candidates['top_hits']['hits']['hits']:
            match_hash = hit['_source']['hash']
            match_offset = hit['_source']['offset']
            possible_offsets = scene_fingerprints[scene_fingerprints[:, 1] == match_hash][:,0] - match_offset
            freq = calc_freq_alt(freq, possible_offsets)
        offset = max(freq, key=freq.get)
        offset_count = freq[offset]
        offset_score = offset_count/candidates['bg_count']
        result.append({'key' : key, 'offset' : offset, 'offset_count' : offset_count,
                       'offset_score' : offset_score })
               
    return result
        
score_with_offset_alt(matches_query, scene_fingerprints)

[{'key': 'DjvCu3QBoaf2Y_2A3bmT',
  'offset': 6,
  'offset_count': 220,
  'offset_score': 0.18410041841004185},
 {'key': 'QpwOvHQBdT-cPgIEckxS',
  'offset': 15,
  'offset_count': 205,
  'offset_score': 0.18141592920353983},
 {'key': '1MtUpXQBXtu81SlS8nsP',
  'offset': 244,
  'offset_count': 1,
  'offset_score': 0.006329113924050633},
 {'key': 'lYJFpHQBOaObx7xaWbv5',
  'offset': 391,
  'offset_count': 1,
  'offset_score': 0.005681818181818182},
 {'key': 'WkEslXQBkSohu0M7zw3h',
  'offset': -18,
  'offset_count': 2,
  'offset_score': 0.0035026269702276708},
 {'key': 'lWGJqXQBE5vHt1SDbQAU',
  'offset': 386,
  'offset_count': 3,
  'offset_score': 0.0027447392497712718},
 {'key': 'epIdqHQB3dj8gDV29z8G',
  'offset': 286,
  'offset_count': 2,
  'offset_score': 0.011627906976744186},
 {'key': 'oR28onQBMd_8Sl4jtDNj',
  'offset': 582,
  'offset_count': 4,
  'offset_score': 0.007561436672967864},
 {'key': '5ErZpXQB1fGddnL2pC6x',
  'offset': -302,
  'offset_count': 5,
  'offset_score': 0.00965250965